In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("data/free_throws.csv")

In [2]:
data.head()

,end_result,game,game_id,period,play,player,playoffs,score,season,shot_made,time
0,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,0 - 1,2006 - 2007,1,11:45
1,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 2 of 2,Andrew Bynum,regular,0 - 2,2006 - 2007,1,11:45
2,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,1,7:26
3,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum misses free throw 2 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,0,7:26
4,106 - 114,PHX - LAL,261031013.0,1.0,Shawn Marion makes free throw 1 of 1,Shawn Marion,regular,21 - 12,2006 - 2007,1,7:18


In [3]:
from statsmodels.stats.weightstats import ztest

new_df = {
    "player": [],
    "regular_mean": [],
    "playoff_mean": [],
    "p_value": []
}

for player, group in data.groupby("player"):
    regular_shots = group[group["playoffs"] == "regular"]["shot_made"].values
    playoff_shots = group[group["playoffs"] == "playoffs"]["shot_made"].values
    
    if len(regular_shots) < 30 or len(playoff_shots) < 30:
        continue
        
    statistic, p_value = ztest(regular_shots, playoff_shots)

    new_df["player"].append(player)
    new_df["regular_mean"].append(np.mean(regular_shots))
    new_df["playoff_mean"].append(np.mean(playoff_shots))
    new_df["p_value"].append(p_value)
    
new_df = pd.DataFrame(new_df)

In [6]:
new_df.sample(10)

,player,regular_mean,playoff_mean,p_value
246,Thaddeus Young,0.687540,0.680000,0.890658
212,Ray Allen,0.908102,0.882353,0.121483
53,Dahntay Jones,0.768903,0.818182,0.356507
102,JaVale McGee,0.583244,0.460000,0.085569
220,Ron Artest,0.724084,0.666667,0.248274
216,Richard Hamilton,0.847518,0.880531,0.194402
92,Goran Dragic,0.748666,0.756757,0.875230
167,Marc Gasol,0.763441,0.792049,0.249039
1,Al Harrington,0.744727,0.660714,0.159277
37,C.J. Miles,0.799242,0.854167,0.350927


In [7]:
new_df[new_df.p_value <= 0.05].shape

(22, 4)

In [8]:
from statsmodels.stats.multitest import multipletests

In [9]:
multipletests(new_df.p_value, alpha=0.05, method='bonferroni')[0].sum()

1

In [10]:
multipletests(new_df.p_value, alpha=0.05, method='holm')[0].sum()

1